<a href="https://colab.research.google.com/github/utkar22/CSE508_Winter2023_Group2_Project/blob/main/Censoring/VOSK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install vosk
!pip install pydub
!apt install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22445 sha256=5b05c177864b403268ff9a40983954f0e09992589d3019a8ee74593a31c38dcc
  Stored in directory: /root/.cache/pip/wheels/69/cb/6a/5e5977c5c1fab3b94ff429718103884855b0d5671ed9880100
Successfully built srt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [2]:
import pandas as pd
import io

from vosk import Model, KaldiRecognizer, SetLogLevel
from pydub import AudioSegment

import wave
from scipy.io import wavfile

from pydub.silence import split_on_silence

In [3]:
import nltk
nltk.download('punkt')  # Download the Punkt tokenizer data
nltk.download('averaged_perceptron_tagger')  # Download the POS tagger data
nltk.download('porter_test')  # Download the Porter stemmer data

from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package porter_test to /root/nltk_data...
[nltk_data]   Unzipping stemmers/porter_test.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/IR_Project

Mounted at /content/drive
/content/drive/MyDrive/IR_Project


In [5]:
def mp3_to_wav(mp3_audio):
    wav_audio = io.BytesIO()
    mp3_audio.export(wav_audio, format="wav")
    return wav_audio

In [6]:
def remove_punct(word):
    new_word = ""
    for i in word:
        if i.isalpha():
            new_word+=i
    return new_word

In [7]:
stemmer = PorterStemmer()

In [8]:
def clean_word(word):
    word = word.lower()
    word = remove_punct(word)
    word = stemmer.stem(word)
    return word

In [9]:
def get_se_dict(se_df):
    full_text = []
    se_dict = {}
    for index, row in se_df.iterrows():
        word = row[2]
        full_text.append(word)
        word = clean_word(word)

        if word not in se_dict:
            se_dict[word] = []
        
        curr = [row[0]*1000,row[1]*1000]
        se_dict[word].append(curr)        

    return se_dict, full_text

In [10]:
def generate_se_dict(path_to_csv):
    se_df = pd.read_csv(path_to_csv, header = None)
    se_dict, full_text = get_se_dict(se_df)
    return se_dict, se_df, full_text

In [11]:
def find_phrase(removal_word, se_df):
    timestamps = []
    phrase = removal_word.split()
    i = 0

    indices = []
    new_indices = []

    for index, row in se_df.iterrows():
        word = row[2]

        if clean_word(phrase[i]) == clean_word(word):
            if i == 0:
                start = row[0]*1000
                ix = index
            i+=1
            if i == len(phrase):
                end = row[1]*1000
                curr = [start,end]
                timestamps.append(curr)
                i=0
                indices.append(ix)
        else:
            i = 0

    for i in indices:
        for j in range(len(phrase)):
            new_indices.append(i+j)

    return timestamps, new_indices

In [12]:
def get_wav_audio(path_to_audio):
    if path_to_audio[-4:] == ".wav":
        audio = AudioSegment.from_wav(path_to_audio)
        return audio
    elif path_to_audio[-4:] == ".mp3":
        audio = AudioSegment.from_mp3(path_to_audio)
        audio_buf = mp3_to_wav(audio)
        audio = AudioSegment.from_wav(audio_buf)
        return audio
    return audio

In [13]:
def remove_word(full_text, removal_word):
    new_indices = []
    for i in range(len(full_text)):
        if clean_word(full_text[i]) == removal_word:
            new_indices.append(i)
    return new_indices

In [22]:
def merge_full_text(full_text):
    output_string = ""
    for i in range(len(full_text)):
        if i>0 and full_text[i-1] == "[REDACTED]" and full_text[i] == "[REDACTED]":
            continue
        output_string+=full_text[i]+" "

    return output_string

In [23]:
def alter_csv(t_df, se_df, indices):
    indices.sort()

    arr = []
    i = 0
    
    for index, row in t_df.iterrows():
        curr = []
        txt = row[3].split()

        for j in txt:
            if i in indices:
                curr.append("[REDACTED]")
            else:
                curr.append(j)
            i+=1

        new_s = merge_full_text(curr)
        arr.append(new_s)

    del t_df[3]

    t_df[3] = arr

    t_df.to_csv('transcript_bleeped.csv', index = False)

In [16]:
def add_bleep(wav_audio, input_arr, se_dict, se_df, full_text):
    timestamps_arr = []
    indices = []

    for removal_word in input_arr:
        if len(removal_word.split())==1:
            removal_word = clean_word(removal_word)
            new_indices = remove_word(full_text, removal_word)
            if removal_word in se_dict:
                timestamps = se_dict[removal_word]
            else:
                timestamps = []
        else:
            timestamps, new_indices = find_phrase(removal_word, se_df)
        timestamps_arr.extend(timestamps)
        indices.extend(new_indices)


    if len(timestamps_arr) == 0:
        return False, False

    bleep_volume = 30

    for i in timestamps_arr:
        start = i[0] - 20
        end = i[1] + 20
        
        
        duration = (end - start)
        
        # Create a bleep sound segment with the same duration as the word segment and apply gain
        bleep_sound = AudioSegment.silent(duration=duration).fade_in(50).fade_out(50)
        bleep_sound = bleep_sound.apply_gain(bleep_volume)
        
        # Overlay the bleep sound over the word segment
        wav_audio = wav_audio.overlay(bleep_sound, position=start, loop=False, gain_during_overlay=-30)

    return wav_audio, indices

In [25]:
def run(path_to_audio, input_string, path_to_csv = "transcript_word.csv", path_to_transcript = "transcript.csv"):
    our_arr = [{"STATUS":None} , {"Name": None}]

    t_df = pd.read_csv(path_to_transcript, header = None)

    se_dict,se_df,full_text = generate_se_dict(path_to_csv)
    wav_audio = get_wav_audio(path_to_audio)

    bleeped_words = input_string.split(',')

    new_audio, indices = add_bleep(wav_audio, bleeped_words, se_dict, se_df, full_text)

    if (new_audio == False):
        our_arr[0]["STATUS"] = False
        return our_arr

    alter_csv(t_df, se_df, indices)

    audio_name = f"audio_files\\{path_to_audio[:-4]}_bleeped.wav"
    new_audio.export(audio_name, format="wav")
    our_arr[0]["STATUS"] = True
    our_arr[1]["Name"] = audio_name
    #our_arr[1]["Audio"] = new_audio

    return our_arr

In [26]:
path_to_audio = "audio.wav"
path_to_csv = "transcript_word.csv"

bleeped_word = "islam, hate"

our_arr = run(path_to_audio, bleeped_word, path_to_csv)

In [20]:
our_arr

[{'STATUS': True}, {'Name': 'audio_files\\audio_bleeped.wav'}]